In [1]:
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing.pool import Pool

from collections import OrderedDict
import sys
import os
print(os.getcwd())
sys.path.insert(0, os.getcwd()+'/...') # +'/..'
sys.path.insert(0, os.getcwd()+'/..') # +'/..'
sys.path.insert(0, os.getcwd()+'/') # +'/..'
from IPython.display import clear_output

/home/sheikh/Motius/Thesis/synch_signal_detection_dl/tools/sionna/examples/myTests


In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer

class IEEE80211Puncturer(Layer):
    def __init__(self, 
                 coding_rate,
                 inverse = False,
                 dtype=tf.float32,
                **kwargs):
        self.coding_rate = coding_rate
        self.inverse = inverse
        if self.coding_rate == (2,3):
            self.puncture_pattern = [1, 1, 1, 0]
            self.punc_size = len(self.puncture_pattern)
            self.punct_vec = [0,1,2]
            self.depunct_vec = [3]
            
            
        if self.coding_rate == (3,4):
            self.puncture_pattern = [1, 1, 1, 0, 0, 1]
            self.punc_size = len(self.puncture_pattern)
            self.punct_vec = [0,1,2,5]
            self.depunct_vec = [3, 4]
        super().__init__(dtype=dtype, **kwargs)
        
        
    def build(self, input_shape):
        
        batch_size = input_shape[0]
        if self.inverse:
            len = np.ceil(input_shape[-1] * self.coding_rate[0] / self.coding_rate[1] * 2).astype(int)
        else:  
            len = input_shape[-1]
        multiples = len//self.punc_size
        p = np.tile(self.punct_vec, (multiples,))
        for ind in range(0,multiples):
          p[ind*(self.punc_size-1): (ind+1)*(self.punc_size-1)] += ind*self.punc_size
        
        for ind in range(0,batch_size):
            [ind, 0,0, p]
        
        self.punct_indices = p
        
            
    def call(self, inputs):
        
         
        if self.inverse:
            new_size = np.ceil(inputs.shape[-1] * self.coding_rate[0] / self.coding_rate[1] * 2).astype(int)
            
            new_shape = [inputs.shape[0],1,1,new_size]
            print(new_shape)
            x_zeros = tf.zeros(new_shape, dtype = tf.int32)
            x = tf.tensor_scatter_nd_update(x_zeros, self.punct_indices, inputs)
        
        else:
          x = tf.gather(inputs, self.punct_indices, axis = 3)
        
        return x
        
        
puncturer = IEEE80211Puncturer((2,3), inverse = False)
depuncturer = IEEE80211Puncturer((2,3), inverse = True)
s = tf.random.uniform([48,], minval=0, maxval=2, dtype=tf.int32)
for i in range(0,3): 
    s = tf.expand_dims(s,0)
s_punct = puncturer(s)
s_depunct = depuncturer(s_punct)


[1, 1, 1, 48]
<tf.Variable 'ieee80211_puncturer_19/Variable:0' shape=(3, 3, 3) dtype=float64, numpy=
array([[[0.32927615, 0.09217954, 0.66069694],
        [0.18971189, 0.20090196, 0.90092036],
        [0.16665378, 0.27839011, 0.81983945]],

       [[0.37511749, 0.07575905, 0.49691476],
        [0.0186322 , 0.35189747, 0.33036034],
        [0.18501296, 0.96041919, 0.00854174]],

       [[0.82652586, 0.9086641 , 0.38298798],
        [0.47282102, 0.50266285, 0.74088155],
        [0.3837641 , 0.72941429, 0.72968289]]])>
<tf.Variable 'ieee80211_puncturer_19/Variable:0' shape=() dtype=float32, numpy=2.881828>
<tf.Variable 'ieee80211_puncturer_19/Variable:0' shape=(3, 3, 3) dtype=float64, numpy=
array([[[0.83533714, 0.95612562, 0.7204703 ],
        [0.52104644, 0.07965491, 0.79836562],
        [0.37687042, 0.24605883, 0.53429377]],

       [[0.21899553, 0.28085621, 0.05070457],
        [0.19338561, 0.48531953, 0.99027024],
        [0.39350208, 0.55572679, 0.7665761 ]],

       [[0.90906785, 0

2023-01-31 04:57:56.947272: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at strided_slice_op.cc:105 : INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1,1,1,1,36], [1,1,1,1,36], and [1] instead.
2023-01-31 04:59:57.839648: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at strided_slice_op.cc:105 : INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1,1,1,1,36], [1,1,1,1,36], and [1] instead.


InvalidArgumentError: Exception encountered when calling layer "ieee80211_puncturer_19" "                 f"(type IEEE80211Puncturer).

{{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:CPU:0}} Inner dimensions of output shape must match inner dimensions of updates shape. Output: [1,1,1,48] updates: [1,1,1,36] [Op:TensorScatterUpdate]

Call arguments received by layer "ieee80211_puncturer_19" "                 f"(type IEEE80211Puncturer):
  • inputs=tf.Tensor(shape=(1, 1, 1, 36), dtype=int32)

In [33]:

s
s_punct = puncturer(s)
s_punct

<tf.Tensor: shape=(1, 1, 1, 48), dtype=int32, numpy=
array([[[[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
          1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
          0, 0, 1, 1, 0, 1, 1, 0]]]], dtype=int32)>

In [34]:
s_punct = puncturer(s)
s_punct

<tf.Tensor: shape=(1, 1, 1, 36), dtype=int32, numpy=
array([[[[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
          0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1]]]], dtype=int32)>

In [24]:
   
s_depunct = depuncturer(s_punct)
s_depunct

TypeError: Exception encountered when calling layer "ieee80211_puncturer_1" "                 f"(type IEEE80211Puncturer).

Too many arguments were given. Expected 3 but got 4.

Call arguments received by layer "ieee80211_puncturer_1" "                 f"(type IEEE80211Puncturer):
  • inputs=tf.Tensor(shape=(1, 1, 1, 36), dtype=int32)